In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
import numpy as np
import pandas as pd
import lightgbm as lgb
import gc

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [4]:
print('Loading data ...')

train = pd.read_csv('../input/train_2016.csv', parse_dates=["transactiondate"])
prop = pd.read_csv('../input/properties_2016.csv')




In [6]:
for c, dtype in zip(prop.columns, prop.dtypes):	
    if dtype == np.float64:		
        prop[c] = prop[c].astype(np.float32)

In [7]:
df_train = train.merge(prop, how='left', on='parcelid')
df_train['transaction_month'] = df_train['transactiondate'].dt.month

In [10]:
df_train.columns.to_series().groupby(df_train.dtypes).groups

In [11]:
x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'], axis=1)
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)

In [12]:
train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

del df_train; gc.collect()

In [13]:
split = 90000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]
x_train = x_train.values.astype(np.float32, copy=False)
x_valid = x_valid.values.astype(np.float32, copy=False)

In [14]:
d_train = lgb.Dataset(x_train, label=y_train)
d_valid = lgb.Dataset(x_valid, label=y_valid)

In [15]:
params = {}
params['max_bin'] = 10
params['learning_rate'] = 0.0021 # shrinkage_rate
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'l2'          # or 'mae'
params['sub_feature'] = 0.5      # feature_fraction 
params['bagging_fraction'] = 0.85 # sub_row
params['bagging_freq'] = 40
params['num_leaves'] = 512        # num_leaf
params['min_data'] = 500         # min_data_in_leaf
params['min_hessian'] = 0.05     # min_sum_hessian_in_leaf

In [16]:
watchlist = [d_valid]
clf = lgb.train(params, d_train, 1, watchlist)

del d_train, d_valid; gc.collect()
del x_train, x_valid; gc.collect()

In [17]:
sample = pd.read_csv('../input/sample_submission.csv')
sample['parcelid'] = sample['ParcelId']

In [18]:
df_test = sample.merge(prop, on='parcelid', how='left')

df_test.head()

In [19]:
train_columns_y = train_columns.drop(['transaction_month'])

In [21]:
#del sample, prop; gc.collect()


#df_test['transaction_month'] = 10
x_test = df_test[train_columns_y]

x_test.head()

In [14]:

df_test['transaction_month'] = 11
y_test = df_test[train_columns]
y_test.head()

In [15]:
df_test['transaction_month'] = 12
z_test = df_test[train_columns]
z_test.head()

In [ ]:
x_test.head()

In [ ]:
#del df_test; gc.collect()
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)
x_test = x_test.values.astype(np.float32, copy=False)

In [ ]:
x_test.head()